In [27]:
import os.path

def doMergeDict(dict1, dict2):
    mergeDict = dict1
    for key in dict1.keys():
        if(not key in dict2.keys()):
            mergeDict[key] = dict1[key]
        else:
            mergeDict[key] = np.append(mergeDict[key], dict2[key])
    for key in dict2.keys():
        if(not key in dict1.keys()):
            mergeDict[key] = dict2[key]
    return mergeDict

class season:
    def calcDayPlayed(self):
        """ Calculate the played matchdays with help of the json-files.
        """
        matchday = 1
        while(os.path.exists(str(self.season)+'/'+str(matchday)+'.json')==True): 
            matchday +=1
        return matchday-1

    def loadTeams(self):
        ''' Return a array with the names of the teams which played in this season.
        '''
        data = leauge.loadMatchDay(self, self.season, 1)
        teams = []
        for x in range(len(data)):
            teams = np.append(teams, ([data[x]['Team1']['TeamName'], data[x]['Team2']['TeamName']]))
        return teams
    
    def loadScores(self, matchDay):
        data = leauge.loadMatchDay(self, self.season, matchDay)
            
        homeResults = dict()
        awayResults = dict()
        homeHalftime = dict()
        awayHalftime = dict()
    
        for x in range(len(data)):
            homeResults[data[x]['Team1']['TeamName']] = data[x]['MatchResults'][1]['PointsTeam1']
            awayResults[data[x]['Team2']['TeamName']] = data[x]['MatchResults'][1]['PointsTeam2']
            homeHalftime[data[x]['Team1']['TeamName']] = data[x]['MatchResults'][0]['PointsTeam1']
            awayHalftime[data[x]['Team2']['TeamName']] = data[x]['MatchResults'][0]['PointsTeam2']
        
        return dict(zip(['homeResults', 'awayResults', 'homeHalftime', 'awayHalftime'], [homeResults, awayResults, homeHalftime, awayHalftime]))
    
    def scoresSeason(self):
        Scores = dict()
        Scores['awayHalftime'] = self.loadScores(1)['awayHalftime']
        Scores['awayResults'] = self.loadScores(1)['awayResults']
        Scores['homeHalftime'] = self.loadScores(1)['homeHalftime']
        Scores['homeResults'] = self.loadScores(1)['homeResults']
        for x in range(self.currentMatchDay-1):
            Scores['awayHalftime'] = doMergeDict(Scores['awayHalftime'], self.loadScores(x+1)['awayHalftime'])
            Scores['awayResults'] = doMergeDict(Scores['awayResults'], self.loadScores(x+1)['awayResults'])
            Scores['homeHalftime'] = doMergeDict(Scores['homeHalftime'], self.loadScores(x+1)['homeHalftime'])
            Scores['homeResults'] = doMergeDict(Scores['homeResults'], self.loadScores(x+1)['homeResults'])
        return Scores
    
    
    def loadPaarings(self, matchDay):
        '''
        Als numpy array weil man darin super nach Begegungen suchen kann ... Beispiel:
        a = np.array([[1,2],[3,4]])
        [1,2] == a
        array([[ True,  True],
               [False, False]], dtype=bool)
        '''
        data = leauge.loadMatchDay(self, self.season, matchDay)
        parings = np.array([[[data[0]['Team1']['TeamName']], [data[0]['Team2']['TeamName']]]])
        
        for x in range(len(data)-1):
            parings = np.append(parings, [[[data[x+1]['Team1']['TeamName']], [data[x+1]['Team2']['TeamName']]]], axis=0)
            
        return np.squeeze(parings)

    def paaringsSeason(self):
        matchPaarungen = [self.loadPaarings(1)]
        for x in range(self.currentMatchDay):
            matchPaarungen = np.append(matchPaarungen, [self.loadPaarings(x+1)], axis=0)
        return np.squeeze(matchPaarungen)
        
    def __init__(self, season):    
        self.season = season
        self.currentMatchDay = self.calcDayPlayed()   # Return the last completed matchday
        self.teams = self.loadTeams()                 # list of all teams which played in this season
        self.scores = self.scoresSeason()
        self.matchPaarungen = self.paaringsSeason()
            # Ein array an welchen Spieltach welche manschaft gegen Welche gespielt hatt.
    
        
exec(open('Einlesen.py').read())

In [28]:
season2016 = season(2016)
pprint(season2016.scoresSeason())
#season2016.loadScores(30)
#Begegnungen laden und nach einer bestimmten suchen
#a = season2016.loadPaarings(30)
#print(a)
#b = ['1. FC Köln','TSG 1899 Hoffenheim']
#b == a

{'awayHalftime': {'1. FC Köln': array([0, 1, 0, 0, 0, 0, 0, 1, 0, 3, 1, 0, 1, 1, 0]),
                  '1. FSV Mainz 05': array([1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 2]),
                  'Bayer 04 Leverkusen': array([0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0]),
                  'Bayern München': array([0, 0, 1, 2, 0, 2, 0, 1, 2, 0, 0, 1, 0, 0, 0]),
                  'Borussia Dortmund': array([0, 2, 0, 0, 3, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1]),
                  'Borussia Mönchengladbach': array([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 2]),
                  'Eintracht Frankfurt': array([0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]),
                  'FC Augsburg': array([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]),
                  'FC Ingolstadt 04': array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 2, 0]),
                  'FC Schalke 04': array([0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0]),
                  'Hamburger SV': array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0,

In [3]:
season2016.matchPaarungen == ['Bayern München']

array([[[ True, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False]],

       [[ True, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False]],

       [[False,  True],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False]],

       [[False, False],
        [False, False],
        [False, False],
        [False, False],
        [ True, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False]],

       [[False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        

In [131]:
dict1 = {'team': 'bvb', 'players': 3}
dict1['blabla'] = 1
dict1['FUU'] = 1
dict2 = {'team': 'bvb', 'players': 3}
dict2['BAZZZ'] = 3
keys1 = dict1.keys()

def doMergeDict(dict1, dict2):
    mergeDict = dict1
    for key in dict1.keys():
        if(not key in dict2.keys()):
            mergeDict[key] = dict1[key]
        else:
            mergeDict[key] = np.append(mergeDict[key], dict2[key])
    for key in dict2.keys():
        if(not key in dict1.keys()):
            mergeDict[key] = dict2[key]
    return mergeDict

doMergeDict(dict1, dict2)

{'BAZZZ': 3,
 'FUU': 1,
 'blabla': 1,
 'players': array([3, 3]),
 'team': array(['bvb', 'bvb'], 
       dtype='<U3')}